Reference : https://jump.dev/JuMP.jl/stable/tutorials/linear/knapsack/

In [1]:
using JuMP
import HiGHS

### Data

In [4]:
n = 8;
capacity = 102.0;
profit = [15.0, 100.0, 90.0, 60.0, 40.0, 15.0, 10.0, 1.0];
weight = [2.0, 20.0, 20.0, 30.0, 40.0, 30.0, 60.0, 10.0];

### Formulation

In [5]:
model = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [6]:
@variable(model, x[1:n], Bin)
@objective(model, Max, sum(profit[i] * x[i] for i in 1:n))
@constraint(model, sum(weight[i] * x[i] for i in 1:n) <= capacity)
print(model)

In [7]:
optimize!(model)
solution_summary(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
1 rows, 8 cols, 8 nonzeros
1 rows, 7 cols, 7 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   1 rows
   7 cols (7 binary, 0 integer, 0 implied int., 0 continuous)
   7 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   331             -inf                 inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   331             265               24.91%        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      280
  Dual bound        280
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    280 (objective)
               

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 2.80000e+02
  Objective bound    : 2.80000e+02
  Relative gap       : 0.00000e+00

* Work counters
  Solve time (sec)   : 1.26815e-03
  Simplex iterations : 2
  Barrier iterations : -1
  Node count         : 1


In [8]:
items_chosen = [i for i in 1:n if value(x[i]) > 0.5]
println("Items chosen: ", items_chosen)

Items chosen: [1, 2, 3, 4, 6]


### Wrting a function

In [9]:
function solve_knapsack_problem(;
    profit::Vector{Float64},
    weight::Vector{Float64},
    capacity::Float64,
)
    n = length(weight)
    # The profit and weight vectors must be of equal length.
    @assert length(profit) == n
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[1:n], Bin)
    @objective(model, Max, profit' * x)
    @constraint(model, weight' * x <= capacity)
    optimize!(model)
    @assert termination_status(model) == OPTIMAL
    @assert primal_status(model) == FEASIBLE_POINT
    
    println("Objective is: ", objective_value(model))
    println("Solution is:")
    for i in 1:n
        print("x[$i] = ", round(Int, value(x[i])))
        println(", c[$i] / w[$i] = ", profit[i] / weight[i])
    end
    chosen_items = [i for i in 1:n if value(x[i]) > 0.5]
    return return chosen_items
end

solve_knapsack_problem(; profit = profit, weight = weight, capacity = capacity)

Objective is: 280.0
Solution is:
x[1] = 1, c[1] / w[1] = 7.5
x[2] = 1, c[2] / w[2] = 5.0
x[3] = 1, c[3] / w[3] = 4.5
x[4] = 1, c[4] / w[4] = 2.0
x[5] = 0, c[5] / w[5] = 1.0
x[6] = 1, c[6] / w[6] = 0.5
x[7] = 0, c[7] / w[7] = 0.16666666666666666
x[8] = 0, c[8] / w[8] = 0.1


5-element Vector{Int64}:
 1
 2
 3
 4
 6